In [1]:
sorted(sc.parallelize([3, 1, 2, 2, 1, 2, 2], 2).countByValue().items())

In [2]:
rdd = sc.parallelize([(3, 1) , (2, 2), (1, 2), (2, 3), (2, 3)])
print sorted(rdd.countByKey().items())
print sorted(rdd.countByValue().items())

In [3]:
from operator import mul
sc.parallelize([1, 2, 3, 4, 5]).fold(1, mul)

In [4]:
rdd = sc.parallelize([1, 2, 3, 4, 5, 6], 3)
print rdd.getNumPartitions()
print rdd.getStorageLevel()
print rdd.glom().collect()

In [5]:
df = spark.read.csv('/FileStore/tables/C2ImportFamRelSample.csv', header=True, inferSchema=True)

In [6]:
df.printSchema()

In [7]:
df.show(10)

In [8]:
from pysprak import SparkContext
spark = SparkContext.builder.nameApp("apc").getOrCreate()
df = spark.read.csv('/FileStore/tables/ParentTeacherConference.csv', header=True, inferSchema=True)

In [9]:
df.cov('Category', 'Maximum Seats')

In [10]:
df.describe()['Category', 'Maximum Seats'].show()

In [11]:
df.filter("Category==2")['Start Time', 'End Time', 'Location'].count()

In [12]:
df.printSchema()

In [13]:
import pyspark.ml.feature as ft

start_dt = ft.StringIndexer(inputCol='Start Date', outputCol='start_dt')
start_tm = ft.StringIndexer(inputCol='Start Time', outputCol='start_tm')
end_dt = ft.StringIndexer(inputCol='End Date', outputCol='end_dt')
end_tm = ft.StringIndexer(inputCol='End Time', outputCol='end_tm')
event_tt = ft.StringIndexer(inputCol='Event Title', outputCol='event_tt')
event_ds = ft.StringIndexer(inputCol='Event Description', outputCol='event_ds')
event_ad = ft.StringIndexer(inputCol='All Day Event', outputCol='event_ad')
end_nt = ft.StringIndexer(inputCol='No End Time', outputCol='end_nt')
Contact_ad = ft.StringIndexer(inputCol='Contact', outputCol='Contact_ad')
Contact_em = ft.StringIndexer(inputCol='Contact Email', outputCol='Contact_em')
Contact_ph = ft.StringIndexer(inputCol='Contact Phone', outputCol='Contact_ph')
Location_ad = ft.StringIndexer(inputCol='Location', outputCol='Location_ad')
Registration_ad = ft.StringIndexer(inputCol='Registration', outputCol='Registration_ad')
last_dr = ft.StringIndexer(inputCol='Last Date To Register', outputCol='last_dr')


In [14]:
featurec = ft.VectorAssembler(inputCols=['start_dt', 
'start_tm', 
'end_dt', 
'end_tm', 
'event_tt', 
'event_ds', 
'event_ad',  
'end_nt', 
'Contact_ad', 
'Contact_em', 
'Contact_ph', 
'Location_ad', 
'Category',                                         
'Registration_ad', 
'last_dr'
], outputCol='features')

In [15]:
import pyspark.ml.classification as cl
logistic = cl.LogisticRegression(maxIter=10, regParam=0.01, labelCol='Maximum Seats')


In [16]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[start_dt, 
                            start_tm, 
                            end_dt, 
                            end_tm, 
                            event_tt, 
                            event_ds, 
                            event_ad,  
                            end_nt, 
                            Contact_ad, 
                            Contact_em, 
                            Contact_ph, 
                            Location_ad, 
                            Registration_ad, 
                            last_dr,                        
                            featurec,
                            logistic])

In [17]:
hr_data_train, hr_data_test = df.randomSplit([0.7,0.3],seed=100)

In [18]:
%sh nc -vz db_hostname db_port

In [19]:
%fs ls

In [20]:
%fs 

In [21]:
%fs ls '/FileStore/tables/'

In [22]:
rdd = spark.read.text('/FileStore/tables/sample_movielens_ratings.txt').rdd

In [23]:
print rdd.collect()
splitdata = rdd.map(lambda row:row.value.split("::"))

In [24]:
print splitdata.collect()
from pyspark.sql import Row
rdd1 = splitdata.map(lambda d:Row(id= int(d[0]), movieid= int(d[1]), rating= int(d[2]), time= long(d[3]) ))

In [25]:
print rdd1.collect()
df = spark.createDataFrame(rdd1)

In [26]:
df.show(5)


In [27]:
df.printSchema()

In [28]:
import pyspark.sql.types as typ
df.withColumn('id', df['id'].cast(typ.IntegerType()))
#df.withColumn('movieid', df['movieid'].cast(typ.IntegerType()))
#df.withColumn('rating', df['rating'].cast(typ.IntegerType()))


In [29]:
df.show()

In [30]:
rdd2 = spark.read.text('/FileStore/tables/Beckman_Sample.txt').rdd

In [31]:
rdd2.take(5)

In [32]:
from pyspark.sql import Row
rdd3 = rdd2.map(lambda row: row.value.split('\t'))
rdd4 = rdd3.map(lambda x: Row(mmid = str(x[0]), did = str(x[2]), typeid = str(x[2]), amtvalue = float(x[3]), areapk = int(x[4]), hightpk = int(x[5])))

In [33]:
rdd4.collect()

In [34]:
pkdf = spark.createDataFrame(rdd4)

In [35]:
pkdf.show()

In [36]:
pkdf.printSchema()

In [37]:
pkdf.columns

In [38]:
pkdf.corr('areapk', 'hightpk', method='pearson')

In [39]:
pkdf.describe().show()

In [40]:
pkdf.distinct().show()

In [41]:
pkdf.dtypes

In [42]:
pkdf.explain()

In [43]:
pkdf.filter('areapk==4788').show()

In [44]:
pkdf.where('hightpk==2236').show()

In [45]:
pkdf.schema

In [46]:
#print pkdf.storageLevel
pkdf.cache().storageLevel

In [47]:
pkdf.toPandas()

In [48]:
pkdf.filter(pkdf.mmid.endswith('_MM2')).show()

In [49]:
from pyspark.sql import functions as F
pkdf.select(pkdf.areapk, F.when(pkdf.did=='No', 1).otherwise(0)).show()

In [50]:
rdd = sc.parallelize(["a", "a", "b", "c"])
rdd.map(lambda x:(x, 1)).collect()

In [51]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import ChiSqSelector
df = spark.createDataFrame(
  [(Vectors.dense([1.0, 2.0, 18.0, 2.0]), 1.0),
   (Vectors.dense([19.0, 22.0, 48.0, 12.0]), 2.0),
   (Vectors.dense([11.0, 21.0, 28.0, 23.0]), 3.0),
   (Vectors.dense([31.0, 32.0, 38.0, 42.0]), 4.0)    
  ],["features", "label"]
)
selector = ChiSqSelector(numTopFeatures=1, outputCol="selectedFeatures")
model = selector.fit(df)
model.transform(df).head().selectedFeatures

In [52]:
out.columns

In [53]:
out[['selectedFeatures']].show()